<h1>Course Selection and Teacher Assignment System

1.**Database Class Initialization**<br>
<small>**Purpose:** Manages the database connection, creates tables, and commits changes to the SQLite database.</h6><br>
**Code:**


**2. init_db Method**<br>
<small>**Purpose:** Creates five tables (students, teachers, semesters, courses, and feedback) if they don't already exist.</h6><br>
**Code:**




In [ ]:
class Database:
    def __init__(self):
        self.connection = sqlite3.connect("university.db")
        self.cursor = self.connection.cursor()
        self.init_db()

    def init_db(self):
      self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS students (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL
        )
    ''')

      self.cursor.execute('''
      CREATE TABLE IF NOT EXISTS teachers (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                rating REAL DEFAULT 0.0,
                research_projects TEXT,
                patents TEXT,
                background TEXT
            )
        ''')

      self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS semesters (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                semester_number TEXT NOT NULL
            )
        ''')

      self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS courses (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                type TEXT CHECK(type IN ('theory', 'lab')),
                teacher_id INTEGER,
                semester_id INTEGER,
                FOREIGN KEY(teacher_id) REFERENCES teachers(id),
                FOREIGN KEY(semester_id) REFERENCES semesters(id)
            )
        ''')

      self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS feedback (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                student_id INTEGER,
                teacher_id INTEGER,
                comment TEXT,
                FOREIGN KEY(student_id) REFERENCES students(id),
                FOREIGN KEY(teacher_id) REFERENCES teachers(id)
            )
        ''')

      self.connection.commit()

    def close(self):
        self.connection.close()

*Each table has specific columns and constraints:<br>*
**students:** <small>Stores student ID and name.</small><br>
**teachers:** <small>Stores teacher details, including name, rating, research projects, patents, and background</small>.<br>
**semesters:** <small>Stores semester numbers.</small><br>
**courses:** <small>Stores course details, type, and links to the teacher and semester.</small><br>
**feedback:** <small>Allows students to give feedback to teachers.</small>

**3.CourseSelectionSystem Class**<br>
**Purpose:** <small>Handles all actions within the system, such as adding students, teachers, semesters, courses, assigning courses, and viewing feedback.</small><br>
**Code:**

In [45]:
class CourseSelectionSystem:
    def __init__(self):
        self.db = Database()

    def add_student(self):
      name = input("Enter student name: ")
      self.db.cursor.execute("INSERT INTO students (name) VALUES (?)", (name,))
      self.db.connection.commit()
      print(f"Student '{name}' added successfully.")

    def add_teacher(self):
        name = input("Enter teacher name: ")
        rating = float(input("Enter teacher rating: "))
        research_projects = input("Enter research projects: ")
        patents = input("Enter patents: ")
        background = input("Enter academic background: ")
        self.db.cursor.execute(
            "INSERT INTO teachers (name, rating, research_projects, patents, background) VALUES (?, ?, ?, ?, ?)",
            (name, rating, research_projects, patents, background))
        self.db.connection.commit()
        print(f"Teacher '{name}' added successfully.")

    def add_semester(self):
      semester_number = input("Enter semester number: ")
      if int(semester_number) > 2:
        print("Error: You can enter only 2 semesters ")
      else:
        self.db.cursor.execute("INSERT INTO semesters (semester_number)VALUES (?)", (semester_number,))
        self.db.connection.commit()
        print(f"Semester '{semester_number}' added successfully.")



    def add_course(self):
      name = input("Enter course name: ")
      course_type = input("Enter course type (theory/lab): ")
      teacher_id = int(input("Enter teacher ID: "))
      semester_id = int(input("Enter semester ID: "))
      self.db.cursor.execute(
            "INSERT INTO courses (name, type, teacher_id, semester_id) VALUES (?, ?, ?, ?)",
            (name, course_type, teacher_id, semester_id))
      self.db.connection.commit()
      print(f"Course '{name}' added successfully.")

    def assign_courses(self):
      student_id = int(input("Enter student ID to assign courses: "))
      semester_id = int(input("Enter semester ID: "))

      theory_courses = input("Enter theory course IDs (comma-separated): ").split(',')
      lab_courses = input("Enter lab course IDs (comma-separated): ").split(',')

      theory_courses = [int(course.strip()) for course in theory_courses]
      lab_courses = [int(course.strip()) for course in lab_courses]

      if len(theory_courses) > 4 or len(lab_courses) > 2:
        print("Error: You can only select 4 theory courses and 2 lab courses.")
        return

      for course_id in theory_courses + lab_courses:
        self.db.cursor.execute("UPDATE courses SET teacher_id = ? WHERE id = ?", (student_id, course_id))

      self.db.connection.commit()
      print("Courses assigned successfully.")

    def submit_feedback(self):
      student_id = int(input("Enter your student ID: "))
      teacher_id = int(input("Enter teacher ID: "))
      comment = input("Enter your feedback: ")
      self.db.cursor.execute("INSERT INTO feedback (student_id, teacher_id, comment) VALUES (?, ?, ?)",
                               (student_id, teacher_id, comment))
      self.db.connection.commit()
      print("Feedback submitted successfully.")

    def view_feedback(self):
      self.db.cursor.execute("SELECT * FROM feedback")
      feedback = self.db.cursor.fetchall()

      print("Feedback:")
      for row in feedback:
        print(f"Student ID: {row[1]}, Teacher ID: {row[2]}, Comment: {row[3]}")

    def view_teacher_profiles(self):
      self.db.cursor.execute("SELECT * FROM teachers")
      teachers = self.db.cursor.fetchall()

      for teacher in teachers:
        print(f"ID: {teacher[0]}, Name: {teacher[1]}, Rating: {teacher[2]}, "
                  f"Research Projects: {teacher[3]}, Patents: {teacher[4]}, Background: {teacher[5]}")

    def view_courses(self):
      self.db.cursor.execute("SELECT * FROM courses")
      courses = self.db.cursor.fetchall()

      for course in courses:
          print(f"ID: {course[0]}, Name: {course[1]}, Type: {course[2]}, Teacher ID: {course[3]}, Semester ID: {course[4]}")

    def view_semesters(self):
      self.db.cursor.execute("SELECT * FROM semesters")
      semesters = self.db.cursor.fetchall()

      for semester in semesters:
          print(f"ID: {semester[0]}, Semester: {semester[1]}")

**4.Method Breakdown**<br><small>
**add_student:** Adds a new student to the students table.</small>

<small>**add_teacher:** <br>Adds a new teacher to the teachers table. The teacher's rating, research projects, patents, and background are also recorded.</small>

<small>**add_semester:** <br>Adds a semester to the semesters table but limits the user to entering only 2 semesters.</small>

<small>**add_course:** <br>Adds a course to the courses table, linking it to a teacher and semester.

<small>**assign_courses:** <br>Allows a student to select up to 4 theory and 2 lab courses. The system ensures that these limits are respected before assigning courses.

<small>**submit_feedback:** <br>Allows students to provide feedback on a teacher. The feedback is stored in the feedback table.

<small>**view_feedback:** <br>Displays all feedback entries from the feedback table.

<small>**view_teacher_profiles:** <br>Displays all teacher profiles stored in the teachers table.

<small>**view_courses:**<br> Displays all courses stored in the courses table.

<small>**view_semesters:** <br>Displays all semester numbers from the semesters table.

**5.Main Program**<br>
**Purpose:** <br><small>The __main__ section creates an interactive console where users can choose between different system operations.</small>

**Code:**

In [46]:
if __name__ == "__main__":
    system = CourseSelectionSystem()
    while True:
        print("\n1. Add Student")
        print("2. Add Teacher")
        print("3. Add Semester")
        print("4. Add Course")
        print("5. Assign Courses")
        print("6. Submit Feedback")
        print("7. View Teacher Profiles")
        print("8. View Courses")
        print("9. View Semesters")
        print("10. View Feedback")
        print("11. Exit")
        choice = input("Enter your choice: ")
        # Perform operations based on user input

        if choice == '1':
            system.add_student()
        elif choice == '2':
            system.add_teacher()
        elif choice == '3':
            system.add_semester()
        elif choice == '4':
            system.add_course()
        elif choice == '5':
            system.assign_courses()
        elif choice == '6':
            system.submit_feedback()
        elif choice == '7':
            system.view_teacher_profiles()
        elif choice == '8':
            system.view_courses()
        elif choice == '9':
            system.view_semesters()
        elif choice == '10':
            system.view_feedback()
        elif choice == '11':
            break
        else:
            print("Invalid choice. Please try again.")



1. Add Student
2. Add Teacher
3. Add Semester
4. Add Course
5. Assign Courses
6. Submit Feedback
7. View Teacher Profiles
8. View Courses
9. View Semesters
10. View Feedback
11. Exit
Enter your choice: 7
ID: 1, Name: Abc, Rating: 5.0, Research Projects: Web, Patents: A, Background: Ph.D.

1. Add Student
2. Add Teacher
3. Add Semester
4. Add Course
5. Assign Courses
6. Submit Feedback
7. View Teacher Profiles
8. View Courses
9. View Semesters
10. View Feedback
11. Exit
Enter your choice: 3
Enter semester number: 2
Semester '2' added successfully.

1. Add Student
2. Add Teacher
3. Add Semester
4. Add Course
5. Assign Courses
6. Submit Feedback
7. View Teacher Profiles
8. View Courses
9. View Semesters
10. View Feedback
11. Exit
Enter your choice: 4
Enter course name: python
Enter course type (theory/lab): theory
Enter teacher ID: 1
Enter semester ID: 1
Course 'python' added successfully.

1. Add Student
2. Add Teacher
3. Add Semester
4. Add Course
5. Assign Courses
6. Submit Feedback
7.

<small>**Exit:** <br>If the user selects "Exit" (option 11), the loop terminates and the database connection is closed.

**6.Database Closure**<br>
<small>
The close method in the Database class is called at the end to close the database connection.

In [ ]:
system.db.close()

**<big>conclusion:</big>** <br>This system simulates a simple university course selection and feedback management system using SQLite. It handles student, teacher, course, and feedback management efficiently through basic SQL operations.